In [8]:
# function definitions are in the script, no definitions in this ipynb allowed!
include("PES_models_bonding_features.jl") # need to reload everytime a change occurs

multirestart (generic function with 1 method)

In [4]:
# test using actual data:
H_data = readdlm("data/h3/h3_data.txt")
# load atomic coordinates:
H_coord = npzread("data/h3/h3_coord.npy")
R = H_data[:,1:end-1]; V = H_data[:, end]
n_data = size(R)[1]

6032

In [5]:
idxes = shuffleobs(1:n_data); # shuffle indexes

In [6]:
# split:
id_train, id_test = splitobs(idxes, at=0.8)
R_train = R[id_train,:]; V_train = V[id_train];
R_test = R[id_test,:]; V_test = V[id_test]
H_coord_train = H_coord[id_train,:,:]; H_coord_test = H_coord[id_test,:,:];

In [218]:
const_r_xy = 1.4172946 # H2
#const_r_xy = 1.9369 #OH+
N = 2 # hyperparam
ρ = f_ρ(R, const_r_xy)
q = f_q_bump(N, ρ)
i = f_i(q) # should be transformed to unique integers
ϵ = f_ϵ(i, q)
# for h:
α = f_α(ϵ)
β = f_β(ϵ)
ub = 1.; lb = -1; θ = rand(2*N+2) .* (ub-lb) .+ lb; # +2 since it starts from 0th index
θ = ones(2*N+2);

In [219]:
n_data, n_d = size(R)
u = similar(R)
w = similar(u)
h = zeros(n_data, n_d, N+1) #stores the first sub primitive
compute_u_w_h!(u, w, h, θ, q, α, β, i, N)
#@benchmark compute_u_w_h!($u, $w, $h, $θ, $q, $α, $β, $i, $N)

In [220]:
x = similar(h)
y = similar(x)
compute_hw_gw!(x, y, w, h, q, N)
#@benchmark compute_hw_gw!($x, $y, $w, $h, $q, $N)

In [10]:
ρ,u,w,x,y = BUMP_feature(θ, R, const_r_xy, N);
@benchmark BUMP_feature($θ, $R, $const_r_xy, $N)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  185.300 μs …   5.798 ms  ┊ GC (min … max):  0.00% … 88.02%
 Time  (median):     234.100 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   301.025 μs ± 305.446 μs  ┊ GC (mean ± σ):  18.44% ± 15.78%

  ▃█▆▄▃▂                                                  ▁     ▁
  ██████▇▆▅▄▃▁▁▃▅█▆▅▅▅▆▄▄▅▅▄▄▁▃▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆███▇▇▇ █
  185 μs        Histogram: log(frequency) by time       1.84 ms <

 Memory estimate: 2.35 MiB, allocs estimate: 22.

In [11]:
b = concat_BUMP(x, y, 5);
@benchmark concat_BUMP($x, $y, 5)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   90.300 μs …   2.146 ms  ┊ GC (min … max):  0.00% … 89.27%
 Time  (median):      94.000 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   123.676 μs ± 152.789 μs  ┊ GC (mean ± σ):  10.73% ±  8.41%

  █     ▁▃▂▁                                                    ▁
  ██▆▆▅▄█████▇▅▅▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃ █
  90.3 μs       Histogram: log(frequency) by time        1.3 ms <

 Memory estimate: 706.94 KiB, allocs estimate: 2.

In [12]:
idxer = atom_indexer(3)
U = f_U_bas_BUMP(idxer, u, w, ρ, e_pow)
@benchmark f_U_bas_BUMP($idxer, $u, $w, $ρ, $e_pow)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   98.900 μs …   5.769 ms  ┊ GC (min … max):  0.00% … 97.80%
 Time  (median):     113.400 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   132.655 μs ± 173.488 μs  ┊ GC (mean ± σ):  12.96% ±  9.46%

  █▃                                                            ▁
  ███▅▇█▅▄▄▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄ █
  98.9 μs       Histogram: log(frequency) by time       1.63 ms <

 Memory estimate: 708.47 KiB, allocs estimate: 50.

In [13]:
Y = f_Y_coord(b, idxer)
@benchmark f_Y_coord($b, $idxer)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  180.100 μs …   5.397 ms  ┊ GC (min … max):  0.00% … 96.15%
 Time  (median):     195.200 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   229.724 μs ± 206.044 μs  ┊ GC (mean ± σ):  12.49% ± 12.29%

  █▅▂                                                        ▁  ▁
  █████▆▄▄▃▃▇▇▆▅▄▅▅▄▁▁▃▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆█▇ █
  180 μs        Histogram: log(frequency) by time       1.55 ms <

 Memory estimate: 1.38 MiB, allocs estimate: 41.

In [16]:
@benchmark Y ./ maximum(abs.(Y))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  118.800 μs …   1.878 ms  ┊ GC (min … max):  0.00% … 90.44%
 Time  (median):     133.900 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   165.837 μs ± 179.053 μs  ┊ GC (mean ± σ):  15.18% ± 12.53%

  █▆▄▂                                                          ▁
  █████▆▅▃▄▄▅▆▆▄▅▄▄▃▃▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆██ █
  119 μs        Histogram: log(frequency) by time       1.34 ms <

 Memory estimate: 1.38 MiB, allocs estimate: 9.

In [21]:
Δ = f_Δcoord(H_coord)
@benchmark f_Δcoord($H_coord)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  104.100 μs …   6.047 ms  ┊ GC (min … max):  0.00% … 97.44%
 Time  (median):     122.500 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   155.996 μs ± 209.649 μs  ┊ GC (mean ± σ):  16.88% ± 11.97%

  █▆▃▁     ▁                                                    ▁
  ████▇▅▆▁██▆▅▅▅▅▅▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▇█ █
  104 μs        Histogram: log(frequency) by time       1.54 ms <

 Memory estimate: 1.24 MiB, allocs estimate: 15.

In [25]:
90549/(6032*3)

5.00381299734748

In [22]:
rk = f_r_orient_vec(b, Δ, idxer)
@benchmark f_r_orient_vec($b, $Δ, $idxer)

BenchmarkTools.Trial: 677 samples with 1 evaluation.
 Range (min … max):  4.984 ms … 27.207 ms  ┊ GC (min … max):  0.00% … 69.68%
 Time  (median):     5.576 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.381 ms ±  3.519 ms  ┊ GC (mean ± σ):  21.90% ± 22.97%

  ██▆▅▄▃▁                           ▂▂▁▁   ▁                  
  ████████▆▇▄▇▄▄▇▄▄▄▅▆▆▁▁▅▁▄▅▁▁▁▇▇█▆████▇█▇█▇█▇▄▄▆▅▄▁▄▆▁▁▄▁▅ ▇
  4.98 ms      Histogram: log(frequency) by time     17.2 ms <

 Memory estimate: 15.60 MiB, allocs estimate: 90549.

In [19]:
@benchmark rk ./ maximum(abs.(rk))

BenchmarkTools.Trial: 3684 samples with 1 evaluation.
 Range (min … max):  769.400 μs … 11.248 ms  ┊ GC (min … max):  0.00% …  0.00%
 Time  (median):       1.067 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.354 ms ±  1.249 ms  ┊ GC (mean ± σ):  22.22% ± 19.47%

  ▇█▇█▅▄▅▁                                         ▁▂▁         ▁
  ████████▆▃▄▄▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▇█████▇▆▆▅▆▅▆▅ █
  769 μs        Histogram: log(frequency) by time      6.89 ms <

 Memory estimate: 4.14 MiB, allocs estimate: 11.

In [23]:
G = f_G_mat(rk)
@benchmark f_G_mat($rk)

BenchmarkTools.Trial: 1367 samples with 1 evaluation.
 Range (min … max):  2.961 ms … 10.965 ms  ┊ GC (min … max):  0.00% … 65.72%
 Time  (median):     3.163 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.654 ms ±  1.067 ms  ┊ GC (mean ± σ):  13.01% ± 17.50%

  ▂▇█▇▅▄▂                               ▁▁▂▃▂▁ ▁              
  ███████▇▆▅▅▅▅▁▄▅▁▄▄▁▁▄▁▁▄▁▄▄▁▁▄▅▅▁▁▁▁▄████████▆▆▆▇▇▆▅▅▅▁▄▅ █
  2.96 ms      Histogram: log(frequency) by time     6.73 ms <

 Memory estimate: 13.12 MiB, allocs estimate: 153.

In [206]:
Φ = f_Φ(U, Y, G, 59)
@benchmark f_Φ($U, $Y, $G, 59)

BenchmarkTools.Trial: 2144 samples with 1 evaluation.
 Range (min … max):  1.139 ms … 16.316 ms  ┊ GC (min … max):  0.00% … 60.67%
 Time  (median):     1.850 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.329 ms ±  2.125 ms  ┊ GC (mean ± σ):  21.69% ± 19.08%

     █▃▁                                                      
  ▄▃▃███▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▂▂▂▃▂ ▂
  1.14 ms        Histogram: frequency by time        11.3 ms <

 Memory estimate: 8.15 MiB, allocs estimate: 2.

In [328]:
#tchebyshev:
C, R_h, R_low, R_0, R_m,R_up, R_C = (1., 0.01, 0.02, 0.5, .9, 10., 11.)
ub = 1.; lb = -1.
#C, R_h, R_low, R_0, R_m,R_up, R_C = rand(7).* (ub-lb) .+ lb;
n_atom, n_basis, g, e = (3, 59, 6, 3)
r_xy, N, max_deg = (1.4172946, 5, 5)
U = f_U_bas(R, idxer, C, R_h, R_C, R_0, g)
println("U ",mean(U), " ", minimum(U), " ", maximum(U)," ", std(U))
b = f_b_feature(R, R_up, R_m, R_low, max_deg, e)
println("b ",mean(b), " ", minimum(b), " ", maximum(b)," ", std(b))
Y = f_Y_coord(b, idxer)
println("Y ",mean(Y), " ", minimum(Y), " ", maximum(Y)," ", std(Y))
Δ = f_Δcoord(H_coord)
rk = f_r_orient_vec(b, Δ, idxer)
println("rk ",mean(rk), " ", minimum(rk), " ", maximum(rk)," ", std(rk))
G = f_G_mat(rk)
println("G ",mean(G), " ", minimum(G), " ", maximum(G)," ", std(G))
Φ = f_Φ(U, Y, G, n_basis)
println("Φ ", mean(Φ), " ", minimum(Φ), " ", maximum(Φ)," ", std(Φ))

U -0.5797520405478458 -1.0 0.0 0.2900095253731425
b 0.01701748041385712 -0.25 0.76041202189571 0.059169315456611256
Y 0.03403496082771424 -0.3832001580107737 0.7611893352040209 0.08443718931310261
rk -0.008908869062954163 -0.5636557216268155 0.5367028157760589 0.0525513751194236
G 0.004023272507118442 -0.1428144911012107 0.3194900981574427 0.018029137279815796
Φ -0.008275105587526695 -1.0 0.7611893352040209 0.09293407838228224


In [358]:
#BUMP:
# hyperparam:
N = 2; max_deg = 5; e_pow = 3; n_basis = 59; const_r_xy = 1.4172946; 
ub = 1.; lb = -1.; θ = rand(2*N+2) .* (ub-lb) .+ lb;
Θ = rand(n_basis, 6) .* (ub-lb) .+ lb;
ρ,u,w,x,y = BUMP_feature(θ, R, const_r_xy, N);
U = f_U_bas_BUMP(idxer, u, w, ρ, e_pow)
println("U ",mean(U), " ", minimum(U), " ", maximum(U)," ", std(U))
b = concat_BUMP(x, y, max_deg);
println("b ",mean(b), " ", minimum(b), " ", maximum(b)," ", std(b))
Y = f_Y_coord(b, idxer)
Y = Y ./ maximum(abs.(Y))
println("Y ",mean(Y), " ", minimum(Y), " ", maximum(Y)," ", std(Y))
Δ = f_Δcoord(H_coord)
rk = f_r_orient_vec(b, Δ, idxer)
rk = rk ./ maximum(abs.(rk))
println("rk ",mean(rk), " ", minimum(rk), " ", maximum(rk)," ", std(rk))
G = f_G_mat(rk)
#G = G ./ maximum(abs.(G))
println("G ",mean(G), " ", minimum(G), " ", maximum(G)," ", std(G))
Φ = f_Φ(U, Y, G, n_basis)
#Φ = Φ./maximum(abs.(Φ))
println("Φ ",mean(Φ), " ", minimum(Φ), " ", maximum(Φ)," ", std(Φ))

U 0.17446762098153076 -1.0 0.6645603457001372 0.17504937067490542
b 0.19917250000016573 -0.27741450205575025 0.9999999999971216 0.3512228536714614
Y 0.19917268302859983 -0.2773363656679622 1.0 0.32475124560286583
rk 0.009894944977976943 -0.958809741264912 1.0 0.07816680420165806
G 0.005252716778684095 -0.03597578058302146 1.000027646724492 0.037276237975188094
Φ 0.04822399163953965 -1.0 1.0000552942133254 0.14208278850915093


In [326]:
@btime rk ./ maximum(abs.(rk));
@btime Y ./ maximum(abs.(Y));
@btime G ./ maximum(abs.(G));
@btime Φ./maximum(abs.(Φ));

  754.900 μs (11 allocations: 4.14 MiB)
  91.500 μs (9 allocations: 1.38 MiB)
  912.600 μs (11 allocations: 6.90 MiB)
  2.397 ms (9 allocations: 16.29 MiB)


6032×59×3 Array{Float64, 3}:
[:, :, 1] =
  1.12042e-6  0.0          0.0         0.0         …   0.0         0.0
  9.97653e-7  0.0          0.0         0.0             0.0         0.0
  1.09305e-6  0.0          0.0         0.0             0.0         0.0
  1.09305e-6  0.0          0.0         0.0             0.0         0.0
  9.40725e-7  0.0          0.0         0.0             0.0         0.0
  9.40725e-7  0.0          0.0         0.0         …   0.0         0.0
  8.17953e-7  0.0          0.0         0.0             0.0         0.0
  7.23381e-7  0.0          0.0         0.0             0.0         0.0
  7.88398e-7  0.0          0.0         0.0             0.0         0.0
  7.88398e-7  0.0          0.0         0.0             0.0         0.0
  7.88398e-7  0.0          0.0         0.0         …   0.0         0.0
  6.65625e-7  0.0          0.0         0.0             0.0         0.0
  6.65625e-7  0.0          0.0         0.0             0.0         0.0
  ⋮                                 

In [9]:
idxer = atom_indexer(3)
N = 2; max_deg = 5; e_pow = 3; n_basis = 59; const_r_xy = 1.4172946; 
ub = 1.; lb = -1.; θ = rand(2*N+2) .* (ub-lb) .+ lb;
Θ = rand(n_basis, 6) .* (ub-lb) .+ lb;
V = f_pot_bond_BUMP(θ, Θ, R, H_coord, idxer, const_r_xy, n_basis, N, e_pow, max_deg)
display(V)
@benchmark f_pot_bond_BUMP($θ, $Θ, $R, $H_coord, $idxer, $const_r_xy, $n_basis, $N, $e_pow, $max_deg)

6032×1 Matrix{Float64}:
 -0.8569920857223543
 -0.8317348505016204
 -0.8589657693380995
 -0.8289011093782888
 -0.7743863730155671
 -0.7641418758882746
 -0.7593882246580776
 -0.7786436725077661
 -0.7303672802089959
 -0.7321024453977021
 -0.7433144666822109
 -0.7620786986682404
 -0.7784694562652015
  ⋮
 -3.280017631174896
 -3.310090412693029
 -3.778974835247613
 -3.7480866136213917
 -2.4650824839501606
 -2.54093001131163
 -2.4341251926178007
 -2.5534968246589824
 -2.5264452378350164
 -2.5820407262497795
 -2.510109150606542
 -2.5396736261502526

BenchmarkTools.Trial: 273 samples with 1 evaluation.
 Range (min … max):  13.314 ms … 40.343 ms  ┊ GC (min … max):  0.00% … 26.55%
 Time  (median):     18.478 ms              ┊ GC (median):    24.45%
 Time  (mean ± σ):   18.326 ms ±  3.058 ms  ┊ GC (mean ± σ):  20.51% ± 11.34%

       ▁            ▅███▂                                      
  █▃▃▃██▆▄▃▃▄▄▂▁▂▂▅██████▆▄▇▄▆▃▅▄▄▂▄▂▃▁▂▃▂▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂ ▃
  13.3 ms         Histogram: frequency by time        28.5 ms <

 Memory estimate: 51.01 MiB, allocs estimate: 90930.